In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

# Data Pipeline & High-Level Workflow

In previous tutorials, we manually iterated the solver step-by-step (`solver.step()`). 
For production runs, analysis, or parameter sweeps, it is more efficient to use the **High-Level API** (`solver.solve()`) along with **Callbacks**.

This tutorial demonstrates:
1.  **Robust Configuration**: Setting up a stable simulation for Earth-scale physics.
2.  **Mock Data Ingestion**: How one might map external data to the solver.
3.  **The `solve()` API**: Running a simulation with automated time-stepping and monitoring.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from cubed_sphere.solvers import CubedSphereSWE, SWEConfig

## 1. Robust Configuration

Stability on the Cubed-Sphere is governed by the CFL condition, which scales with $\frac{1}{N^2}$.
For global simulations (Earth Radius $R \approx 6371$ km), the time step ($dt$) is automatically chosen based on the maximum wave speed to ensure stability.

> **Feature Highlight: Dynamic DG CFL Time-Stepping**
>
> We are proud to introduce our **Dynamic DG CFL Time-Stepping** engine. You no longer need to guess a stable `dt` or manually calculate the CFL condition. The solver automatically analyzes the grid structure, polynomial order ($N$), and the instantaneous flow field to compute the optimal stable time step for every iteration.


In [ ]:
# Physical Constants (Earth)
R_earth = 6.37122e6  # Earth Radius (m)
g = 9.80616          # Standard Gravity (m/s^2)
H_avg = 10000.0      # Average Ocean/Atmosphere Depth (10km)
N = 16               # Polynomial Order. Higher N supports finer wave structures.

# --- Understanding Time-Step Limitations ---
# The solver computes 'dt' automatically, but it's useful to understand the physics:
# 1. Gravity Wave Speed: c = sqrt(gH) ~ 313 m/s
c_wave = np.sqrt(g * H_avg)

# 2. Approximate Flow Velocity (e.g. Jet Stream) ~ 40 m/s
u_flow_max = 40.0 

# 3. Characteristic Speed
v_max_total = c_wave + u_flow_max

print(f"Gravity Wave Speed: {c_wave:.1f} m/s")
print(f"Max Characteristic Speed: {v_max_total:.1f} m/s")
print("The Dynamic DG engine uses this speed + grid spacing to set dt.")

# Initialize Config 
target_cfl = 0.5

config = SWEConfig(
    N=N, 
    R=R_earth, 
    gravity=g, 
    H_avg=H_avg,
    CFL=target_cfl,
    backend='numpy', # Standard CPU backend
    # We do NOT set 'dt'. The solver handles it.
)

solver = CubedSphereSWE(config)


## 2. Data Ingestion (Mock Scenario)

In a real pipeline, you would load weather data (e.g., GFS, ECMWF) on a Lat/Lon grid and interpolate it to the Cubed-Sphere faces.

Here, we simulate this process by "loading" the **Williamson Case 2** initial condition.

In [ ]:
# Mock: Create a source Lat/Lon grid (Conceptual)
# In production, you would load this from NetCDF/GRIB files.
lat_source = np.linspace(-90, 90, 180)
lon_source = np.linspace(0, 360, 360)
LAT, LON = np.meshgrid(lat_source, lon_source)
# data_source = load_from_netcdf(...) 

# For this tutorial, we generate the clean Case 2 state directly.
# This ensures we start with a physically balanced state.
initial_state = solver.get_initial_condition(type="case2")

# The state is stored in the Conserved Form (h*sqrt_g, hu*sqrt_g, hv*sqrt_g).
# This is crucial for conservation properties during the flux computation.
print(f"Initial State Shape: {initial_state.shape} (Vars, Face, Xi, Eta)")


## 3. High-Level Execution: `solver.solve()`

Instead of writing a manual `for` loop, we use `solve()`. This method handles the time-integration loop and accepts **Callbacks** for monitoring or I/O.

In [ ]:
# Define a simple Monitor Callback
def simple_monitor(t, state):
    """
    Monitor callback executed at specified intervals.
    
    Args:
        t: Current simulation time
        state: Current system state (Conservative variables)
    """
    # Quick Conservation Check (Proxy)
    # State[0] is Mass variable (h * sqrt_g).
    # To get true total mass, we must integrate: sum(state[0] * w_alpha * w_beta).
    # However, for a quick stability check, the raw sum is sufficient to detect numerical explosions (NaNs).
    current_mass_proxy = np.sum(state[0])
    
    # Print status to stdout
    print(f"[Monitor] Time: {t:.1f}s | Mass Proxy: {current_mass_proxy:.4e}")

# Define Simulation Span (e.g., 1 Hour)
t_hours = 1.0
t_final = t_hours * 3600.0
# The solver will take as many steps as needed to reach t_final safely
t_span = (0.0, t_final)

print(f"Starting Simulation for {t_hours} hour(s)...")

# Execute Solver
# We pass our custom callback to monitor progress.
# The solver uses the 'Dynamic DG CFL' engine to determine 'dt'.
final_state = solver.solve(t_span, initial_state, callbacks=[simple_monitor])

print("Simulation Complete.")


## 4. Post-Processing and Visualization

With the simulation complete, we visualize the final Geopotential Height ($h$).

In [ ]:
# Extract Height Field
# Post-Processing: Convert from Conserved Variables to Physical Primitive Variables.
face_idx = 0
face_name = list(solver._impl.topology.FACE_MAP)[face_idx]
sqrt_g = solver._impl.faces[face_name].sqrt_g

# Physics: h = (h * sqrt_g) / sqrt_g
h_field = final_state[0, face_idx] / sqrt_g

plt.figure(figsize=(10, 8))
# Plotting in degrees (approximate for visualization)
plt.imshow(h_field, origin='lower', extent=[-45, 45, -45, 45], cmap='viridis')
plt.colorbar(label='Geopotential Height (m)')
plt.title(f"Final Height Field (Face {face_idx}) @ T={t_final/3600:.1f}h")
plt.xlabel('Xi (Degrees)')
plt.ylabel('Eta (Degrees)')
plt.show()
